# Example of NetCDF slicing with xarray
The package "xarray" provides a high level interface to NetCDF files. For CDO-like operations on large datasets that do not fit in memory, the package "dask" is required.

In [1]:
import pandas as pd
import xarray as xr
import dask # check if dask is installed (not used directly)

For this example, we use the 1990-2014 precipitation file used to force LISFLOOD over Europe (approximately 30 GB):

In [2]:
path = '/net/ies-mpd01/vol/H01_Fresh_Water/Europe/forcing_data/JRC_5km_meteo_forcing_netcdf/chunking_20x20x20_uncompressed/pr.nc'
nc = xr.open_dataset(path, chunks={'time': 100})

If a file is open using the "chunks" keyword argument, operations can be declared on datasets that do not fit in memory relying on dask lazy evaluation (http://xarray.pydata.org/en/stable/dask.html#). The specified chunks must fit in the computer's memory.

To see the data variable's shape and dimensions:

In [3]:
print nc.pr.shape
print nc.pr.dims

(9131, 950, 1000)
(u'time', u'y', u'x')


Here we set the slicing masks along the "x" and "y" dimensions, given some lower and upper bounds (for indexing methods, see: http://xarray.pydata.org/en/stable/indexing.html):

In [4]:
mask_y = xr.ufuncs.logical_and(nc.y >= 2992500, nc.y <= 2997500)
mask_x = xr.ufuncs.logical_and(nc.x >= 5002500, nc.x <= 5007500)
sliced_pr = nc.pr[:,mask_y,mask_x]

The slicing is actually performed when the declared sliced variable (sliced_pr) is written to a NetCDF file:

In [5]:
t0 = pd.datetime.now()
sliced_pr.to_netcdf('sliced_pr.nc') # the actual computing is performed here
print 'The slicing took {:.2f} seconds'.format((pd.datetime.now() - t0).total_seconds())

The slicing took 257.65 seconds


The newly created NetCDF file contains only 1 data variable (pr) with its attributes and dimensions. The missing variable (lambert_azimuthal_equal_area) and global file attributes can be copied from the source file:

In [6]:
new_nc = xr.open_dataset('sliced_pr.nc')
new_nc['lambert_azimuthal_equal_area'] = nc['lambert_azimuthal_equal_area']
new_nc.attrs = nc.attrs
new_nc.to_netcdf('sliced_pr_with_metadata.nc')

Other operations can be declared on large datasets and evaluated lazily using xarray+dask, e.g.:
* group-by: http://xarray.pydata.org/en/stable/groupby.html
* resample: http://xarray.pydata.org/en/stable/generated/xarray.DataArray.resample.html?highlight=resample

A lower-level interface to NetCDF files is provided by the netCDF4 package (http://unidata.github.io/netcdf4-python/).